In [ ]:
#-----------------------------
# Importing relevant libraries
#-----------------------------

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import cv2
import os

import numpy as np
import csv
import matplotlib.pyplot as plt

In [ ]:
#-----------------------------------------------
# Importing data from the CSV in relevant format
#-----------------------------------------------
with open("fer2013.csv") as f:  #change data file to fer2013.csv
    content = f.readlines()

lines = np.array(content)

num_of_instances = lines.size

x_train = []
y_train = []
x_test = []
y_test = []

for i in range(1,num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")  
        val = img.split(" ")    
        pixels = np.array(val, 'float32')
        emotion = tf.keras.utils.to_categorical(emotion, 7)
    
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
        print("",end="")

In [ ]:
#data transformation for train and test sets
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

In [ ]:
#------------------------------------------
# CNN MODEL which has 2 Convolution Layers
#------------------------------------------
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(48,48,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=30)

val_loss, val_acc = model.evaluate(x_test, y_test) 
print(val_loss) 
print(val_acc*100)

train_loss, train_acc = model.evaluate(x_train, y_train) 
print(train_loss) 
print(train_acc*100)

In [ ]:
model.save('my_model.h5')

In [ ]:
model = load_model('my_model.h5')

In [ ]:
#---------------------------
# Confusion Matrix
#---------------------------
from sklearn.metrics import classification_report, confusion_matrix
 
pred_list = []; actual_list = []
predictions = model.predict(x_test)
 
for i in predictions:
    pred_list.append(np.argmax(i))
 
for i in y_test:
    actual_list.append(np.argmax(i))
 
confusion_matrix(actual_list, pred_list)

In [ ]:
#---------------------------
#Predicting for Custon Image
#---------------------------

img = cv2.imread('IMG_20170519_121506 (2).jpg')
img = cv2.resize(img,(48,48))
img = np.resize(img, [1,48,48,1])

emotion = model.predict(img)


objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
y_pos = np.arange(len(objects))
    
plt.bar(y_pos, emotion[0], align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('percentage')
plt.title('emotion')
    
plt.show()